In [7]:
import pandas as pd
import ta

In [8]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [9]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band

ta.volatility.AverageTrueRange
symbols = ['ETH']


nstd = 10

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)


df.dropna(inplace=True)

In [10]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_price = 0

        self.sl = 0

        self.band_stop_run = 0.02     # <= 1
        self.slm = 3



        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.9994      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

        self.cross_up = {}
        self.cross_down = {}
        self.sma_cross_up = {}
        self.sma_cross_down = {}
        self.reset_crosses([symbols])

    def buy_long(self, symbol, long_price, atr, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.long_price = long_price
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        self.sl = long_price - (env.slm*atr)
        print('filled long', long_price, time, '\n\n')

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (env.slm*atr)
        print('filled short', short_price, time, '\n\n')

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price*self.trading_fee_multiplier) - sell_price)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    

    def reset_crosses(self, symbols):
        for symbol in self.symbols:
            self.cross_up[symbol] = False
            self.cross_down[symbol] = False
            self.sma_cross_up[symbol] = False
            self.sma_cross_down[symbol] = False


In [11]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under lower boll. band
        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long

    Short:
        1. Check if symbol crosses over upper boll. band
        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short

SELL:
    Long:
        1. Check if symbol crosses over upper boll. band
        2. Check if close is under stop loss; if so, sell
        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long
    
    Short:
        1. Check if symbol crosses under lower boll. band
        2. Check if close goes over stop loss; if so, sell
        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short
"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under lower boll. band\n        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long\n\n    Short:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short\n\nSELL:\n    Long:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if close is under stop loss; if so, sell\n        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long\n    \n    Short:\n        1. Check if symbol crosses under lower boll. band\n        2. Check if close goes over stop loss; if so, sell\n        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short\n'

In [12]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if env.cross_down[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i],time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.cross_down[symbol] = True
    

            ######## SHORT
            if env.cross_up[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] <= (df[f'{symbol}_Upper_Band'].iloc[i] - ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
                env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
                env.cross_up[symbol] = True



    ####################################################################################################################

    ############## Want to sell long position
    elif env.balance_unit != 'BUSD':

        # Sell long at break-even  
        if env.sma_cross_up[env.balance_unit] == True and df[f'{env.balance_unit}-USD_Low'].iloc[i] <= env.long_price and df[f'{env.balance_unit}-USD_High'].iloc[i] >= env.long_price:
            print('Break even hit: sell long', env.long_price, df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_long(sell_price=env.long_price, time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue


        # Sell long for profit
        if env.cross_up[env.balance_unit] == True and df[f'{env.balance_unit}-USD_Close'].iloc[i] <= (df[f'{symbol}_Upper_Band'].iloc[i] - ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at stop loss
        elif df[f'{env.balance_unit}-USD_Close'].iloc[i] <= env.sl:
            print('Stop loss hit: sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        

        # Check if SMA is crossed up
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_SMA'].iloc[i]:
            print('SMA long crossed')
            env.sma_cross_up[env.balance_unit] = True

        # Check if upper band is crossed
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_Upper_Band'].iloc[i]:
            print('long crossed')
            env.cross_up[env.balance_unit] = True
    

    ############## Want to sell short position
    else:
        print(env.short_amount)

        # Sell short at break-even  
        if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price and df[f'{env.short_unit}-USD_Low'].iloc[i] <= env.short_price:
            print('Break even hit: sell short', env.short_price, df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_short(sell_price=env.short_price, time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Sell short for profit
        if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at stop loss
        elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
            print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        

        # Check if SMA is crossed down
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
            print('SMA short crossed')
            env.sma_cross_down[env.short_unit] = True

        # Check if lower band is crossed
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
            print('short crossed')
            env.cross_down[env.short_unit] = True





if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0

KeyboardInterrupt: 

In [ ]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2021-07-03 18:00:00,2021-07-14 09:15:00,2021-07-18 02:10:00,2021-07-30 04:45:00,2021-08-15 13:05:00,2021-09-03 04:50:00,2021-10-14 07:15:00,2021-10-16 13:10:00,2021-10-26 10:05:00,2021-11-13 09:05:00,2021-11-24 03:35:00,2021-12-29 18:25:00,2022-01-12 01:35:00,2022-01-29 15:35:00,2022-02-04 00:20:00,2022-02-10 08:35:00,2022-02-23 22:00:00
1,2199.4,1976.26,1963.6,2356.02,3123.76,3890.19,3727.53,3858.68,4155.29,4658.12,4313.59,3637.57,3241.64,2630.95,2768.91,3166.64,2472.33
2,long,short,long,long,long,short,short,long,long,short,short,long,short,short,short,long,long
3,2021-07-03 19:00:00,2021-07-14 15:05:00,2021-07-18 04:00:00,2021-07-30 06:40:00,2021-08-16 20:20:00,2021-09-03 10:30:00,2021-10-15 12:55:00,2021-10-16 15:45:00,2021-10-26 11:35:00,2021-11-13 13:35:00,2021-11-24 03:55:00,2021-12-29 21:00:00,2022-01-12 04:00:00,2022-01-29 16:00:00,2022-02-04 08:35:00,2022-02-10 10:00:00,2022-02-24 00:35:00
4,2199.4,1976.26,1963.6,2356.02,3123.76,4027.3,3866.96,3858.68,4155.29,4658.12,4313.59,3637.57,3241.64,2630.95,2799.83,3166.64,2316.61
